# parse human sequence from fasta

In [3]:
from Bio import SeqIO

# Nuskaityti seką iš fasta failo
fasta_file = "NP_000468.1.fa"  # Pakeiskite į savo failo kelią
sequences = SeqIO.parse(fasta_file, "fasta")

# Išspausti visas sekas
for seq_record in sequences:
    print(f"ID: {seq_record.id}")
    print(f"Description: {seq_record.description}")
    print(f"Sequence: {seq_record.seq}")
    sequence = seq_record.seq


ID: NP_000468.1
Description: NP_000468.1 albumin preproprotein [Homo sapiens]
Sequence: MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLPKLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTKVHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPADLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKCCAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVSTPTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKATKEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAALGL


# Blast human sequence and write results to xml

In [ ]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Blast import NCBIXML

# Atlikti BLAST paiešką
result_handle = NCBIWWW.qblast("blastp", "swissprot", sequence, entrez_query='Mammalia[Organism]')

# Išsaugoti rezultatus į failą
with open("blast_results.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()


In [ ]:
# Atidaryti ir nuskaityti BLAST rezultatus
with open("blast_results.xml") as result_handle:
    blast_records = NCBIXML.parse(result_handle)
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            print("**** Alignment ****")
            print("Sequence:", alignment.title)
            for hsp in alignment.hsps:
                print("E-value:", hsp.expect)
                print(hsp.query)
                print(hsp.match)
                print(hsp.sbjct)

# Blast human sequence and save filtered results to .fasta file

In [4]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import Entrez

# Settings
query_sequence = sequence  # Replace with your sequence
output_file = "matches.fasta"  # File to save the matching sequences

# Perform BLAST search
# result_handle = NCBIWWW.qblast("blastp", "swissprot", query_sequence)
result_handle = NCBIWWW.qblast("blastp", "swissprot", sequence, entrez_query='"Mammalia"[Organism]', megablast=True)

In [8]:

with open("blast_results.xml") as result_handle:
    blast_records = NCBIXML.parse(result_handle)
    # Process the BLAST results
    with open(output_file, "w") as out_handle:
        # blast_records = NCBIXML.parse(result_handle)
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                # hsp = alignment.hsps[0]                                                                                                                 norint maziau duomenu, kad nebutu pasikartojanciu seku
                for hsp in alignment.hsps:
                    alignment_length = alignment.length
                    # match_length = hsp.align_length
                    identity = hsp.identities / alignment_length
                    # alignment_coverage = hsp.align_length / len(query_sequence)
                    if identity >= 0.0:
                    # if alignment_coverage >= 0.8:
                        sequence_id = alignment.accession  # Get the sequence ID
                        Entrez.email = "jaunius.tamulevicius@mif.stud.vu.lt"  # Enter your email here
                        handle = Entrez.efetch(db="protein", id=sequence_id, rettype="fasta")
                        record = SeqIO.read(handle, "fasta")
                        SeqIO.write(record, out_handle, "fasta")  # Write the sequence to file
                        handle.close()

result_handle.close()


# Align matched sequences with mafft

In [ ]:
!apt-get install mafft  # Įdiegiamas MAFFT programa

# Nurodykite kelias iki savo fasta failo
fasta_file_path = "matches.fasta"

# Atlikti sekų palyginimą su MAFFT
!mafft --auto --thread -1 $fasta_file_path > aligned_sequences.fasta


# Construct phylogenetic tree

In [ ]:
from Bio import AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Phylo import write

# Nurodykite kelias iki jūsų surenkamų sekų (fasta formato)
aligned_sequences_file = "aligned_sequences.fasta"

# Nuskaityti sekas iš failo
alignment = AlignIO.read(aligned_sequences_file, "fasta")

# Sukurti skaičiuotuvą atstumui
calculator = DistanceCalculator('identity')

# Apskaičiuoti atstumą tarp sekų
distance_matrix = calculator.get_distance(alignment)

# Sukurti medžio sudarymo konstruktorių
constructor = DistanceTreeConstructor()

# Sudaryti medį pagal UPGMA metodą
upgma_tree = constructor.upgma(distance_matrix)

# Įrašyti medį į failą
output_tree_file = "phylogenetic_tree.nwk"
write(upgma_tree, output_tree_file, "newick")

# Find the most unique fragments in human sequence from provided sequence list

In [5]:
from Bio import SeqIO
from pprint import pprint

# Path to your fasta file
# fasta_file_path = "aligned_sequences_unfiltered.fasta"
fasta_file_path = "aligned_sequences.fasta"

# Read sequences from the file
sequences = list(SeqIO.parse(fasta_file_path, "fasta"))

# Find the human sequence
human_sequence = None
for seq_record in sequences:
    if "human" in seq_record.description.lower():
        human_sequence = str(seq_record.seq)
        break

if human_sequence:
    fragment_scores = {}

    # Iterate through the human sequence by one symbol step
    for i in range(len(human_sequence) - 14):
        fragment = human_sequence[i:i+15]
        score = 0

        # Check each symbol in the fragment for uniqueness
        for symbol_idx, symbol in enumerate(fragment):
            if symbol == "-":
                score = 1000
                continue
            symbol_count = sum(symbol in str(seq_record.seq)[i+symbol_idx] for seq_record in sequences)
            score += symbol_count
        
        fragment_scores[fragment] = score

     # Sort fragment_scores by score in ascending order
    sorted_dict_by_values = dict(sorted(fragment_scores.items(), key=lambda item: item[1]))
    sorted_dict_by_values_reverse = dict(sorted(fragment_scores.items(), key=lambda item: item[1], reverse=True))
    # print(fragment_scores.items())

    print(f"sorted_dict_by_values: {sorted_dict_by_values}")
    # print(f"sorted_dict_by_values_reverse: {sorted_dict_by_values_reverse}")

    # Find the fragment with the lowest score
    most_unique_fragment = min(fragment_scores, key=fragment_scores.get)
    least_unique_fragment = max(fragment_scores, key=fragment_scores.get)
    print(f"The most unique 15-amino acid fragment in the human sequence is: {most_unique_fragment}")
    # print(f"The least unique 15-amino acid fragment in the human sequence is: {least_unique_fragment}")
else:
    print("Human sequence not found in the file.")


sorted_dict_by_values: {'LGEENFKALVLIAFA': 71, 'GEENFKALVLIAFAQ': 71, 'EENFKALVLIAFAQY': 71, 'ENFKALVLIAFAQYL': 71, 'NFKALVLIAFAQYLQ': 71, 'FFAKRYKAAFTECCQ': 71, 'FKDLGEENFKALVLI': 72, 'PELLFFAKRYKAAFT': 72, 'ELLFFAKRYKAAFTE': 72, 'LLFFAKRYKAAFTEC': 72, 'LFFAKRYKAAFTECC': 72, 'KDLGEENFKALVLIA': 73, 'NLPRLVRPEVDVMCT': 73, 'FAKRYKAAFTECCQA': 73, 'VAHRFKDLGEENFKA': 74, 'NPNLPRLVRPEVDVM': 74, 'PNLPRLVRPEVDVMC': 74, 'CLLPKLDELRDEGKA': 74, 'LLPKLDELRDEGKAS': 74, 'LPKLDELRDEGKASS': 74, 'LDELRDEGKASSAKQ': 74, 'ICTLSEKERQIKKQT': 74, 'AVMDDFAAFVEKCCK': 74, 'EGKKLVAASQAALGL': 74, 'DLGEENFKALVLIAF': 75, 'DNPNLPRLVRPEVDV': 75, 'LPRLVRPEVDVMCTA': 75, 'YAPELLFFAKRYKAA': 75, 'APELLFFAKRYKAAF': 75, 'AKRYKAAFTECCQAA': 75, 'KRYKAAFTECCQAAD': 75, 'ACLLPKLDELRDEGK': 75, 'FHADICTLSEKERQI': 75, 'HADICTLSEKERQIK': 75, 'ADICTLSEKERQIKK': 75, 'DICTLSEKERQIKKQ': 75, 'ATKEQLKAVMDDFAA': 75, 'TKEQLKAVMDDFAAF': 75, 'KEQLKAVMDDFAAFV': 75, 'AHRFKDLGEENFKAL': 76, 'HRFKDLGEENFKALV': 76, 'RFKDLGEENFKALVL': 76, 'FKALVLIAF

# Find the least unique fragments in human sequence from provided sequence list

In [1]:
from Bio import SeqIO
from pprint import pprint

# Path to your fasta file
fasta_file_path = "aligned_sequences_unfiltered.fasta"
# fasta_file_path = "aligned_sequences.fasta"

# Read sequences from the file
sequences = list(SeqIO.parse(fasta_file_path, "fasta"))

# Find the human sequence
human_sequence = None
for seq_record in sequences:
    if "human" in seq_record.description.lower():
        human_sequence = str(seq_record.seq)
        break

if human_sequence:
    fragment_scores = {}

    # Iterate through the human sequence by one symbol step
    for i in range(len(human_sequence) - 14):
        fragment = human_sequence[i:i+15]
        score = 0

        # Check each symbol in the fragment for uniqueness
        for symbol_idx, symbol in enumerate(fragment):
            if symbol == "-":
                continue
            symbol_count = sum(symbol in str(seq_record.seq)[i+symbol_idx] for seq_record in sequences)
            score += symbol_count
        
        fragment_scores[fragment] = score

     # Sort fragment_scores by score in ascending order
    sorted_dict_by_values = dict(sorted(fragment_scores.items(), key=lambda item: item[1]))
    sorted_dict_by_values_reverse = dict(sorted(fragment_scores.items(), key=lambda item: item[1], reverse=True))
    # print(fragment_scores.items())

    # print(f"sorted_dict_by_values: {sorted_dict_by_values}")
    print(f"sorted_dict_by_values_reverse: {sorted_dict_by_values_reverse}")

    # Find the fragment with the lowest score
    most_unique_fragment = min(fragment_scores, key=fragment_scores.get)
    least_unique_fragment = max(fragment_scores, key=fragment_scores.get)
    # print(f"The most unique 15-amino acid fragment in the human sequence is: {most_unique_fragment}")
    print(f"The least unique 15-amino acid fragment in the human sequence is: {least_unique_fragment}")
else:
    print("Human sequence not found in the file.")


sorted_dict_by_values_reverse: {'FQNALLVRYTKKVPQ': 393, 'QNALLVRYTKKVPQV': 393, 'VRYTKKVPQVSTPTL': 390, 'NALLVRYTKKVPQVS': 388, 'CENQDSISSKLKECC': 386, 'RYTKKVPQVSTPTLV': 384, 'ALLVRYTKKVPQVST': 381, 'YLYEIARRHPYFYAP': 380, 'YEIARRHPYFYAPEL': 379, 'LLVRYTKKVPQVSTP': 379, 'YTKKVPQVSTPTLVE': 379, 'EIARRHPYFYAPELL': 377, 'LYEIARRHPYFYAPE': 373, 'LVRYTKKVPQVSTPT': 372, 'DCCAKQEPERNECFL': 371, 'ICENQDSISSKLKEC': 371, 'EYARRHPDYSVVLLL': 371, 'EYKFQNALLVRYTKK': 368, 'ADCCAKQEPERNECF': 367, 'KFQNALLVRYTKKVP': 367, 'FLYEYARRHPDYSVV': 366, 'RRHPDYSVVLLLRLA': 366, 'LGEYKFQNALLVRYT': 366, 'YEYARRHPDYSVVLL': 363, 'YARRHPDYSVVLLLR': 363, 'KYLYEIARRHPYFYA': 362, 'GEYKFQNALLVRYTK': 362, 'TKKVPQVSTPTLVEV': 362, 'QDSISSKLKECCEKP': 360, 'CCAKQEPERNECFLQ': 359, 'NQDSISSKLKECCEK': 359, 'VFLGMFLYEYARRHP': 359, 'LLRLAKTYETTLEKC': 359, 'FLGMFLYEYARRHPD': 358, 'MFLYEYARRHPDYSV': 357, 'LRLAKTYETTLEKCC': 357, 'YICENQDSISSKLKE': 356, 'RRPCFSALEVDETYV': 356, 'RHPDYSVVLLLRLAK': 355, 'DVFLGMFLYEYARRH': 354, 'RPCFSAL

In [17]:


# Calculate and print the length of the dictionary
dict_length = len(fragment_scores)
print(f"The length of the dictionary is: {dict_length}")


The length of the dictionary is: 610
